<a href="https://colab.research.google.com/github/AlexH20/Supervised-ML-sentiment-measures/blob/main/Results_and_Descriptive_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descriptive statistics

In [2]:
%%capture
!pip install tabulate
!pip install pysentiment2
!pip install linearmodels
!pip install stargazer

import pandas as pd
pd.options.mode.chained_assignment = None 
import pysentiment2 as ps
from google.colab import drive 
from tabulate import tabulate
import re
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import statsmodels.api as sm
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

pd.set_option('display.expand_frame_repr', False)
drive.mount("/content/gdrive")

In [31]:
#Data DM methods
data_dm = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woScAR_dm.csv")
data_dm["Date"] = pd.to_datetime(data_dm["Date"], format = "%Y-%m-%d")
data_dm["Date"] = pd.to_datetime(data_dm["Date"])
data_dm["Year"] = [x.year for x in data_dm["Date"]]
data_dm["Month"] = [x.month for x in data_dm["Date"]]
data_dm["ordered_month"] = [((x[1]["Year"]-2015)*12 + x[1]["Month"]) for x in data_dm.iterrows()]
data_dm = data_dm[~data_dm["ordered_month"].isin([0,1,2,3,56,57,58,59,60])]
data_dm = data_dm.fillna(0)

#Data RF
data_rf = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woScAR_RF.csv")
data_rf["Date"] = pd.to_datetime(data_rf["Date"])
data_rf["Year"] = [x.year for x in data_rf["Date"]]
data_rf["Month"] = [x.month for x in data_rf["Date"]]
data_rf["ordered_month"] = [((x[1]["Year"]-2015)*12 + x[1]["Month"]) for x in data_rf.iterrows()]
data_rf = data_rf[~data_rf["Year"].isin([2019])]
data_rf = data_rf[~data_rf["ordered_month"].isin([0,1,2,3])]

#Data BERT
data_BERT = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woScAR_BERT.csv")
data_BERT["Date"] = pd.to_datetime(data_BERT["Date"])
data_BERT["Year"] = [x.year for x in data_BERT["Date"]]
data_BERT["Month"] = [x.month for x in data_BERT["Date"]]
data_BERT["ordered_month"] = [((x[1]["Year"]-2015)*12 + x[1]["Month"]) for x in data_BERT.iterrows()]
data_BERT = data_BERT[~data_BERT["ordered_month"].isin([0,1,2,3,56,57,58,59,60])]

Here are the results of the panel data regression:

In [32]:
data_rf_acc = data_rf[data_rf["AR_RF"] != 0]
acc_rf = (data_rf_acc["AR_dummy"] == data_rf_acc["AR_RF"]).sum() / len(data_rf_acc["AR_dummy"])
data_BERT_acc = data_BERT[data_BERT["AR_BERT"] != 0]
acc_BERT = (data_BERT_acc["AR_dummy"] == data_BERT_acc["AR_BERT"]).sum() / len(data_BERT_acc["AR_dummy"])
print("Accuracy of RF: {}".format(acc_rf))
print("Accuracy of BERT: {}".format(acc_BERT))

Accuracy of RF: 0.5065336394137383
Accuracy of BERT: 0.5029743589743589


In [33]:
data_dm["Size"] = np.log(data_dm["Size"])
data_rf["Size"] = np.log(data_rf["Size"])
data_BERT["Size"] = np.log(data_BERT["Size"])

#HIV dictionary 

endg_var = "AR"
exog_var = ["Nasdaq","Turnover","Size","BTM","pref_alpha", "HIV4_tone"]

pooled_y = data_dm[endg_var]
pooled_x = data_dm[exog_var]

pooled_x = sm.add_constant(pooled_x)

pooled_olsr_model_hiv4 = sm.OLS(endog=pooled_y, exog=pooled_x)
pooled_olsr_model_results_hiv4 = pooled_olsr_model_hiv4.fit()
print(pooled_olsr_model_results_hiv4.summary())


#LM dictionary

endg_var = "AR"
exog_var = ["Nasdaq","Turnover","Size","BTM","pref_alpha", "LM_tone"]

pooled_y = data_dm[endg_var]
pooled_x = data_dm[exog_var]

pooled_x = sm.add_constant(pooled_x)

pooled_olsr_model_lm = sm.OLS(endog=pooled_y, exog=pooled_x)
pooled_olsr_model_results_lm = pooled_olsr_model_lm.fit()
print(pooled_olsr_model_results_lm.summary())

#RF

endg_var = "AR"
exog_var = ["Nasdaq","Turnover","Size","BTM","pref_alpha", "AR_RF"]

pooled_y = data_rf[endg_var]
pooled_x = data_rf[exog_var]

pooled_x = sm.add_constant(pooled_x)

pooled_olsr_model_rf = sm.OLS(endog=pooled_y, exog=pooled_x)
pooled_olsr_model_results_rf = pooled_olsr_model_rf.fit()
print(pooled_olsr_model_results_rf.summary())

#BERT

endg_var = "AR"
exog_var = ["Nasdaq","Turnover","Size","BTM","pref_alpha", "AR_BERT"]

pooled_y = data_BERT[endg_var]
pooled_x = data_BERT[exog_var]

pooled_x = sm.add_constant(pooled_x)

pooled_olsr_model_BERT = sm.OLS(endog=pooled_y, exog=pooled_x)
pooled_olsr_model_results_BERT = pooled_olsr_model_BERT.fit()
print(pooled_olsr_model_results_BERT.summary())





                            OLS Regression Results                            
Dep. Variable:                     AR   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     6.385
Date:                Thu, 09 Jun 2022   Prob (F-statistic):           9.79e-07
Time:                        22:29:32   Log-Likelihood:             2.9266e+05
No. Observations:              110191   AIC:                        -5.853e+05
Df Residuals:                  110184   BIC:                        -5.852e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.786e-06      0.001     -0.003      0.9